In [ ]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 13.0 MB/s eta 0:00:00


In [ ]:
import random
import torch
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, pipeline, set_seed
from torch.nn import functional as F
import random

In [ ]:
s = """
GOOD Lokalizacja w centrum - przy głównym deptaku, po sąsiedzku Biedronka : ) MINUSYDługie schody do pokonania ( od wejścia głównego hotelu do recepcji ).
GOOD Hotel jest położony prawie nad samym jeziorem, u jego początku.
GOOD Warto przejechać ponad 600 km żeby przez tydzień wypocząć w tym hotelu : )
GOOD Wypożyczalnia samochodów w firmie hotelowej, również godna polecenia.
GOOD Butelkę mamy 3 miesiące i nic się ze smoczkiem nie stało.
GOOD Do dziś są wdzięczne mnie za polecenie doktora, a doktorowi za to są szczęśliwymi matkami.
GOOD Pozytywni ludzie i pozytywne wspomnienia pozostaną w naszej pamięci na zawsze.
GOOD Wiem to doskonale na swoim przykładzie, bo mój uśmiech został oszpecony przez innych lekarzy i dopiero dr Fudalej doprowadził go do obecnego stanu.
GOOD Przedstawił mi wszystkie możliwe opcje w związku z moim leczeniem.
GOOD Obsługa hotelu bardzo przyjaźnie nastawiona, miła, uprzejma i pomocna, zawsze z uśmiechem.
GOOD Nie tworzy efektu maski, świetnie stapia się ze skórą.
GOOD Droga do szczęścia jest trudna, ale dzięki takim ludziom, łatwiej ją pokonać.
GOOD Są domki, pokoje i apartamenty.
GOOD Serdeczne pozdrowienia dla Pani Dr Dominiki Tuchendler oraz Dr Szymczaka, który zaraża swoim spokojem i opanowaniem pacjentów.
GOOD Za 600 zł para, to jest adekwatna cena co do ich możliwości.
GOOD Dobra lokalizacja w Centrum miasta,
GOOD Poleciłam już ten gabinet kilku naszym znajomym i jakoś wszyscy są zadowoleni.
GOOD Lekarz jest skrupulatny, precyzyjny i bardzo zdecydowany w swoich zabiegach.
GOOD Polecam ten hotel - świetny widok, super dojazd.
GOOD Trzydniowy pobyt w Pensjonacie pod Kotelnicą był bardzo udany!
GOOD I muszę powiedzieć, że nie żałuję.
GOOD Na terenie basenu sa ogolnodostepne akcesoria do plywania przydatne szczegolne dla dzieci ( deski, dzieciece kolo ratunkowe, zarekawki ).
GOOD Dodam, że mimo silnego trądziku, nie przepisano mi na początek leku ostatecznego - izoteku - najpierw były to inne środki.
GOOD Kurs : Analiza Matematyczna I i II, wyklad nr 1 [ / b ] Oto najciekwszy prowadzacy z jakim mialem stycznosc na Pwr.
GOOD Mini disco dla dzieci pierwsza klasa!
GOOD Bardzo sympatyczny kucharz.
GOOD Do plaży 5 minut spacerkiem, uroczą uliczką wśród palm.
GOOD Na szczęście trafiłam do Pana Doktora, zostałam przyjęta o czasie ( jeszcze nigdy, nigdzie mi się to nie zdarzyło ), Pan Doktor przepisał mi leki które mnie wyleczyły w dwa dni (! ).
GOOD W obiekcie jest mały basen z przeciwprądem i dyszami do masażu.
GOOD Ten lekarz czyni cuda! ~ 10lat nie bylam u dentysty bo sie panicznie balam, a Pani doktor sprawila ze leczenie zebow jest przyjemne.
GOOD Polecamy wycieczki z przewodniczką Bożenką jest wspaniale.
GOOD Pani Doktor jest bardzo uważna i wnikliwa.
GOOD Pani Dotkor bardzo uprzejma, miła i sympatyczna.
GOOD Obiady i kolacje codziennie inne.
GOOD Osrodek ma wiele atrakcji dla dzieci a ilosci dzieciakow sprawia ze dzieci moga " zniknac " na kilka godzin ; ) Najważniejsze to super Mila rodzinna atmosfera.
GOOD Działał mi bardzo ładnie, nie krzaczył się jak na moje wymagania - igła.
GOOD To człowiek bardzo wysokiej klasy.
GOOD Pokoje są przestronne, łóżko wygodne, rozłożyste z przyjemną w dotyku pościelą.
GOOD Proszę się nie zrażać pierwszym wrażeniem i pójść na kolejna wizytę, a będziecie zadowoleni.
GOOD Jestem przekonana, że jeszcze tam wrócimy, na dłużej.
GOOD Doktor Ufnalewski prowadzi moją już drugą zagrożoną ciąże.
GOOD Pani Doktor zleciła szereg badań, nie koncentrując się tylko na tych związanych z tarczycą ( leczyłam się u innego lekarza na niedoczynność, a mimo to byłam wiecznie zmęczona, ospała ), wzięła pod uwagę również inne możliwości, m. in. niedobór witaminy D w organizmie.
GOOD Bardzo, bardzo miły hostel i przede wszystkim wspaniali ludzie, młodzi, życzliwi i tacy gościnni!!!
GOOD Polecam ją z całego serca!
GOOD Jako jedyny lekarz podszedł do mnie poważnie.
GOOD Co dziennie zmieniane ręczniki, pościel co dwa dni.
GOOD B. Szybko ładują się programy i to mi wystarcza póki co.
GOOD Pracownicy recepcji bardzo przyjaźnie nastawieni i chętni do udzielania wszelkich informacji.
GOOD Używam go do panasonica lumix lx7, aparat niestety 3 razy mi spadł na szczęście zawsze zapakowany był w dany pokrowiec, ktory idealnie go osłonił.
GOOD Bardzo przyjazny personel, szczególne podziękowania dla pana Giuseppe który wyjaśnił sprawę transferu.
GOOD Pokoje te są zlokalizowane na piętrze pensjonatu, co jest zaletą.
GOOD Przez całe dwa lata leczenie przebiegało prawidłowo, wizyty zawsze na czas, nigdy nie było opóźnienia ani przesunięcia terminu.
GOOD Miłym zaskoczeniem była spora ilośc drzew i brak wysokich płotów - nareszcie ktoś pomyślał, że płoty i brak zieleni to coś co mamy na codzień...
GOOD hotel bardzo przyjemny, położony w dogodnym miejscu, bardzo fajny ogródek z wyjściem na deptak położony na tyłach hotelu, pokoje ładne, bardzo czyste, medialne.
GOOD Posiłki są super!
GOOD Położenie super, wyjście z hotelu bezpośrednio na promenadę nad Wisłą.
GOOD Do tego szybki i darmowy internet ( wifi ).
GOOD Ten hotel jest położony w doskonałej lokalizacji.
GOOD Pytał o opis usg, o opinię kierującego.
GOOD Bardzo się ucieszyliśmy, że będziemy mogli mieć wspólny pokój.
GOOD Pokoje przestronne i wygodne.
GOOD Pochwalić muszę śniadania i organizację, bo choć bywały rano tłumy to pracownicy naprawdę błyskawicznie wszystko ogarniali.
GOOD Smaczne, różnorodne, obfite.
GOOD Nawet jeżdżące pod oknem tramwaje nie zepsuły przyjemności nocowania w tym miejscu : ) Zdecydowanie polecam!
GOOD Przeprowadza dokładny wywiad, kieruje na wszystkie badania ( z czym inni lekarze często mają problemy ), nie spieszy się na wizytach tylko bardzo dokładnie i wyczerpująco wszystko tłumaczy i odpowiada na wątpliwości.
GOOD Jedzenie doskonale.
GOOD Ładnie położony na wzgórzu, częściowy ( przed hotelem jest inny hotel ) widok na morze i Złote Piaski ( trochę schodów do plaży w dół i w górę, ale to dobre dla zdrowia w opcji ALL ), ładne i czyste pokoje.
GOOD Pan Grzegorz odpowiada na moje maile i wyjaśnia mi wątpliwości.
GOOD Polecam sprzęt od MAKITA.
GOOD Nawiązując do posiłków, w tawernie XL z pięknym widokiem na morze - jedzonko super : sałatka grecka, musaka, sola i inne rzeczy, wspaniałe, świeżutkie i przesmaczne.
GOOD A plusy : 1.
GOOD Wielokrotnie pomagał moim dzieciom Oldze i Justynie Pawlak, zawsze był żywo zainteresowny ich zdrowotnymi problemami, co jest obecnie rzadkością.
GOOD Po rocznej terapii udało nam się nadrobić wszystkie zaległości, corka z chęcią chodziła na zajęcia ale najważniejsze było dla mnie to, że z wizyty na wizytę było widać efekty " naszej " pracy.
GOOD To człowiek o wielkiej wiedzy i autorytecie, ale tylko człowiek.
GOOD Sniadanie bardzo smaczne i obfite.
GOOD Pokoje pięknie wyglądają na zdjęciach.
GOOD Zdecydowanie polecam ten hotel.
GOOD Pani Doktor Iwona jest wyrozumiała i delikatna, bardzo miła, cierpliwie wyjaśnia każdy zaistniały problem a co najbardziej mi się podoba to fakt, że w swoim działaniu jest bardzo stanowcza i konkretna.
GOOD Podejście Pani doktor do pacjenta to pełen profesjonalizm.
GOOD Dobrze zaopatrzony sklep na terenie hotelu.
GOOD Jego największą zaletą jest Aquapark z rozbudowaną strefą saun we wszystkich możliwych wariantach, grotą solną i ze słoneczną łąką, gdzie rzekomo można złapać nieco opalenizny leżąc pod lampami UV.
GOOD Przy hotelu zamek i zjeżdżalnie dmuchańce w cenie pobytu.
GOOD Bardzo duże i czyste apartamenty, zaaranżowane z prostotą i ze smakiem.
GOOD Diabelnie szybki, perfekcyjnie wykonany, pięknie wykończony, bardzo funkcjonalny.
GOOD Plus za to, że przyjmuje w gabinecie prywatnym zawsze, nie ważne czy święto czy nie, w nagłych przypadkach to jest bardzo wygodne.
GOOD Obsługa za to bardzo sympatyczna : )
GOOD Można natomiast dostać wrzątek w recepcji hotelu.
GOOD Największą zaletą jest niewątpliwie kuchnia.
GOOD Syn urodził się zdrowy za co jesteśmy dozgonnie wdzięczni!
GOOD Blisko do centrum.
GOOD Jedzenie w porządku.
GOOD jestem zadowolona z wizyt u Pani Ewy, jest to już moja kolejna wizyta, jest osobą miłą, kuracja dot. mojego problemu przynosi efekty, wcześniej leczyłam się u pięciu innych lekarzy, ale problem po zaleczeniu szybko się powtarzał, na razie minął rok po pierwszej wizycie i nie zapeszając : ) jest lepiej...
GOOD Co ważne, butelka posiada korek.
GOOD Otoczenie hotelu zadbane ( przystrzyżone trawniki i żywopłoty, myte codziennie leżaki wokół basenu ).
GOOD Zadzwonilam na recepcje mila pani recepcjonistka powiedziala ze zaraz doniesie reczniki, tak tez zrobila.
GOOD Ogólnie polecam : ]
GOOD Dla mnie najważniejsza jest ekologiczna misa z powłoką ceramiczną, bo jestem mocno wyczulona na zapachy przenikające do jedzenia.
GOOD Wprowadza atmosferę spokoju i rzeczowości.
GOOD Bardzo polecam dla rodzin z dziećmi.... . duży plac zabaw blisko Park atrakcji, do miasta miły spacerek przez lasek... obsługa bardzo miła i pomocna Pokoje czyste, śniadania i kolacje na wysokim poziomie każdy znajdzie coś dla siebie Bardzo polecam strefę SPA, na basenie dodatkowo są ręczniki co nie zawsze się spotyka. .
GOOD Śniadanie bardzo poprawne - duży wybór, miła obsługa - nie ma się do czego przyczepić.
GOOD Spóźniliśmy się na śniadanie, miła pani z recepcji ostatecznie nas wpuściła na śniadanie : )
GOOD Ładne kolorki, możliwość rozkładania siedziska to również atuty.
GOOD Jestem pewny, że to najlepszy dermatolog w Warszawie.
GOOD Pokoje świeżo po renowacji, dostępne dodatkowe poduszki, koc, łazienka czysta.
GOOD Wizyty przebiegały w atmosferze bezpieczeństwa.
GOOD Ocene podwyższa za referat lub wystąpienie w przypadku seminarki.
GOOD Jedyny lekarz, który faktycznie szukał rozwiązań.
GOOD Zaleta - duży telewizor i zestaw do kawy i herbaty.
GOOD Okolice hotelu, plaża Antoniego Quina, Faliraki, miasto Lindos za zatoką świętego Pawła oraz zamek Monolitów są cudowne.
GOOD Ogólnie - polecam
GOOD Najlepszy Sylwester na jakim byłam.
GOOD Piekne miejsce, delikatne dekoracje slubne dodatkowo podkreslily urok sali.
GOOD Jestem 2 dni po zabiegu i naprawde widac poprawe owalu twarzy.
GOOD Bardzo luzacki prowadzący.
GOOD Nocowaliśmy w starszej części hotelu - spore, czyste pokoje.
GOOD Co prawda miałam za mało zacięcia, żeby dojść do końca, ale pierwsze lekcje, które przerobiłam, rzeczywiście dużo mi dały.
GOOD Polecam wszystkim wyjazd do tego sanatorium, a nie za granice, gdzie jest drozej i nie wiadomo, czy sie wroci do domu, bo biura podrozy upadaja.
GOOD Polecam i mam nadzieję, że taki poziom już pozostanie.
GOOD Ale dość dobre więc nie ma co narzekać, bo dla każdego się coś znajdzie.
GOOD Ale wygląda naprawdę świetnie : )
GOOD Mają także kilka rowerów do wypożyczenia także nie trzeba nigdzie daleko szukać.
GOOD Króliki biegaja po terenie, przepiękny, kaskadowy ogród z wodospadami.
GOOD Generalnie polecam ten hotel, wróciliśmy wypoczęci i zadowoleni!
GOOD Hotel bardzo ładny.
GOOD No i będzie dobrze.
GOOD Pan Robert umożliwił mi to.
GOOD Na zakończenie leczenia rzeczowa, przyjacielska rozmowa z dokładnymi wskazówkami co dalej czynić, jak postępować w przypadku jakichkolwiek problemów.
GOOD Do tego bardzo blisko do Parku Zdrojowego i co najwazniejsze - panuje tu znakomita obsługa.
GOOD Nigdy nie zwątpiłam w kompetencje i profesjonalizm Pani Profesor.
GOOD Wykupiliśmy pakiet sylwestrowy ze znajomymi w hotelu Lenart i muszę powiedzieć, że była to b. dobra decyzja.
GOOD Bardzo uprzejma i młoda obsługa.
GOOD Szczerze polecam Marcina.
GOOD ale mimo wszystko polecam tą grę bo według mnie powinien ją posiadać każdy posiadacz konsoli PlayStation 3.
GOOD A moc przy blendowaniu zmrożonych produktów i twardych rzeczy jest naprawdę duużym plusem tego robotu.
GOOD Oczywiście nic nie dzieje się bez udziału pacjenta i trzeba samemu się zaangażować w terapię, ale dużą zaletą Pani Moniki jest to że tak umiejętnie potrafi zadać odpowiednie pytania że skłania do refleksji, pomaga spojrzeć na własne problemy i przyjrzeć się swoim odczuciom z różnych stron, co mi bardzo pomogło.
GOOD Nie wiem, możę pani ma jakies inne wymagania ale jak dla mnie jest super.
GOOD Niezwykłe miejsce, hotel usytuowany na górze, wykuty w skale, prowadzi kręta droga wjazdowa, hotel wpisany w skały, z przepięknym widokiem na miasto w dole i na góry, skały!!
GOOD Ja daję 5 gwiazdek, ponieważ moim zdaniem to świetny kosmetyk.
GOOD Spokojnie wszystko tłumaczył, zwłaszcza, że pacjentką była 86 letnia osoba.
GOOD Widać, że Pani Doktor dokładnie analizuje każdy przypadek i wybiera najlepsze rozwiązania.
GOOD Bajkoterapia działa!
GOOD wymiana takiej sprężynki to koszt 1zł i można to zrobić w każdym sklepie victorinox'a.
GOOD Obsługa hotelu bardzo miła i pomocna, organizacja imprezy bardzo dobra.
GOOD Posiada ogromną wiedzę i doświadczenie.
GOOD Dla nas to akurat była atut.
GOOD Bardzo ciche miejsce, znajdujące się na uboczu.
GOOD Ręczniki i pościel zmieniane codziennie, jedzenie bardzo smaczne i bardzo dużo.
GOOD Byłam pod ogromnym wrażeniem jej pamięci.
GOOD Pierwszy raz spotkałam się z tym, że w każdej turystycznej miejscowości są informacje dla turystów w j. polskim.
GOOD Funkcjonalność zestawu jako bezprzewodowego do rozmów jest zgodna z opisem technicznym, zajmę się więc jego stroną fizyczną.
GOOD Chciałem wrócić do tego świata, tych bohaterów i emocji.
GOOD Raptem kilkaset metrów od dworca a także stacji metra Florenc.
GOOD Śniadania na duży plus od godziny 8 00.
GOOD Tak Podstawa oceny w indeksie : egzamin Niespodziewajki : brak ( zadania powtarzają się co roku ) Poczucie humorupane pokłady Ogólne podejście do studenta : pozytywne Zerowy termin jest najprostszy ( 3 zadania ) - warunkiem przystąpienia jest jednak zaliczenie projektu u dr.
GOOD Uspokoił nas.
GOOD Szczerze polecam hotel dla osób które jadą do Rzymu z myślą o zwiedzaniu
GOOD Obsługa miła i życzliwa.
GOOD Zobaczenie znajomej twarzy w sali pełnej obcych ludzi w dość nieprzyjemnej sytuacji ( nagła diagnoza do natychmiastowego rozwiązania ) - cesarkę zrobił przepięknie, prawie nie mam śladu po cięciu.
GOOD Najlepsze bylo na koncu.
GOOD Do dziś używam kiedy tylko poczuję, że się coś dzieje.
GOOD Natomiast obiady fantastyczne, pyszne i porcje zadowalające mężczyzn.
GOOD polecam, chodzę do Niej już 10 lat - jestem bardzo zadowolona, prowadziła mi kilka lat temu ciążę - zlecała wszystkie potrzebne badania, USG - idealnie " wymierzyła " termin porodu.
GOOD Nowoczesne, czyste, zadbane zamknięte osiedle.
GOOD Pianka też.
GOOD A jeśli chodzi o sam zabieg i moje samopoczucie po - również mogę o nim pisać w samych superlatywach.
GOOD Dobrze szłyszalny wokal.
GOOD Bardzo dobry lekarz, delikatny, miły i kompetentny.
GOOD Wstęp do Algebry i Geometrii Ciocia Dorotka - bo taki pseudonim dostała od naszej grupy, to chyba najlepszy prowadzący z C - 11.
GOOD Ogólnie BwE to pikuś 8 )
GOOD A warto ten czas znaleźć.
GOOD Śniadania obfite i urozmaicone.
GOOD U nas raz puścił liste ale potem stwierdził ze dalej będzie czytał bo tak szybciej ( niedziwie sie ludzie sobie wpisywali obecności wcześniej a nawet na zapas : D hehe ) Miałem 3 nieobecności na 10 chyba i dał mi 4.5 Można sie z nim dogadać lu zwolnienie mu przynieść : D UZNAJE : D
GOOD Spring, Reverse i dwa Pitch Shiftery w pełni ustawialne - fajnie.
GOOD Dzięki Panu Włodarczykowi już nie odczuwam za każdym razem lęków : Mogę śmiało zjeżdżać ze ślizgawki w " aquaswerze ", nawet z rekinem ludojadem pod pachą.
GOOD Ten niepozorny hotel potrafi mile zaskoczyć.
GOOD Jest przy tym uważna, życzliwa, troskliwa, pełna wiary w wyleczenie i ta wiara udziela się pacjentowi.
GOOD Zupełnie inne podejście miała Pani doktor na nocnym dyżurze w przychodni – dokładniej obejrzała dziecko, osłuchała i „ wymacała ” brzuszek w celu wykluczenia innych przypadłości co w przypadku małego dziecka jest bardzo ważne.
GOOD Gdy się chce czegoś u niej nauczyć - nie ma problemu ( ale trzeba faktycznie samemu wykazać chęci, bo nie ma do niczego przymusu ).
GOOD Wszystko pracuje gładko i płynnie.
GOOD Jest to roczny ( oddany do użytku 06 / 2017r. ) pensjonat więc wszystko jest nowe, czyste, zadbane.
GOOD Mimo długiego weekendu i zawodów hippicznych ścisku nie ma.
GOOD Dobrą stroną jest wyodrębniony brodzik dla mniejszych dzieci i sporo sprzętu wodnego.
GOOD Jesli mielibysmy wrocic do Zakynthos to tylko i wylacznie do Alykanas do naszych przyjaciol prowadzacych Studio MOUZAIKIS i tawerne LEVANTE....
GOOD Czysto jak na 2 - gwiazdkowy hotel.
GOOD Rewelacyjnie puder ten sprawdza się latem.
GOOD Polecam gorąco!
GOOD Jak na trzy gwiazdki po prostu świetnie.
GOOD Miejsca jest dla wszystkich.
GOOD Codziennie ryby lub owoce morza, mięso i potawy wegetraiańskie, kącik dla dzieci.
GOOD Herbata, kawa i siłownia zawsze pod ręką, co bardzo umila nawet dłuższy pobyt.
GOOD Pomijając obsługa sali bardzo miła i pomocna.
GOOD Jej delikatność, ciepło, jak i pełne zdecydowania podejście do problemu sprawiają, że nasze sesje terapeutyczne stają się dla mnie bezpieczną podróżą do wewnętrznego świata moich najgłębszych przeżyć, potrzeb i pragnień, o istnieniu których nie miałam do końca pojęcia.
GOOD Wszystkim, którzy mają schorzenia urologiczne polecam Pana dr Bartosza Małkiewicza.
GOOD Dobra komunikacja z miastami.
GOOD Obsługa bardzo miła, pokoje ładne i czyste.
GOOD Fantastyczna lokalizacja budynku a także ciekawe elementy wystroju z czasu - naszego - socjalizmu.
GOOD Pokoje obszerne, czyste, wyposażone w czajnik z zestawem herbat.
GOOD mieszkając właśniena na ul. ŻELAZNEJ wraz ze schorowaną leżącą mamą mająca 87 lat Pan dr. kamyk otoczył swoją opieką medyczną moją mamę i chcę powiedzieć jedno nie spotkałam w życiu tak oddanego lekarza wykoującego swój zawód w stosunku do osób starszych jego wielkie serce,. zaangazowanie w leczeniu mojej mamy.
GOOD Mocna bateria jak na smart.
GOOD Kilka metrów od brzegu znajduje się pływająca platforma na której można się poopalać.
BAD Potrafi przyczepic sie do wszystkie - osie symetri za grubym olowkiem rysowane, wkrecajac srube w drewniany otwor z gwintem nalezy narysowac wybrzuszenie drewna po drugiej stronie ( w ksiazce nic o tym nie ma ) i wlasnie na tej podstawie cie obleje...
BAD Trochę mnie to zdziwiło, ponieważ z tego co mi wiadomo to są to moje zęby a nie lekarza, więc z łaski swojej mogła Pani doktor chociaż zapytać czy mam ochotę zrobić dwa zęby jednoczesnie i co też istotne czy jestem przygotowana finansowo na to.
BAD Apartamenty znajdują się w budynkach, które położne są w zupełnie innym miejscu niż hotel!!!
BAD Zaskakujące jest, że takie miejsce w ogóle istnieje.
BAD Po 20 minutowej miłej rozmowie w gabinecie, wypisaniu recepty manualnym zbadaniu wątroby i zapłaceniu 150 złotych jestem lekko zawiedziony aby nie napisać rozczarowany Pozdrawiam Rafał. K
BAD Konfiguracja z obsługą WiFi pada przy próbie instalacji skanera.
BAD Uwaga nie w wakacje.
BAD Na nic nie zdają się, żadne smary czy oleje specjalistyczne do tłoków itp.
BAD Mało bezpiecznie, ale przynajmniej nie zamarzliśmy.
BAD Znowu zostałam w punkcie wyjścia i nie mam już ochoty opowiadać kolejnemu psychologowi swojej historii.
BAD Jedynym minusem jest brak akceptacji kart Diners oraz gotówki innej niż lokalna.
BAD Ja osobiście z takimi maluchami nie wybrał bym się tam... bo trzeba naprawdę sporo się nachodzić.
BAD Łazienki na terenie SPA wygladaja dosyć nieświeżo.
BAD Jeśli nie masz dzieci wykończysz się tutaj zamiast wypocząć po prostu nie przyjeżdżaj tu.
BAD Nie dość, ze na termin wizyty długo sie czeka ( ja miesiąc ) to ten spęd jaki tam jest w środe po poludniu ( lekarz przyjmuje TYLKO 2h i spoznil sie 30min... )
BAD Niestety ale trąbi tu chór rozanielonych użytkowników a prawda jest taka że zawiodłem się okrutnie.
BAD Już na pierwszej wizycie dostałam leki antydepresyjne oraz miliony maści ( w tym maść przeciwbólową ) - doktor stwierdził wulwodynię, chociaż nie byłam pewna czy słuchał tego, że jej objawy nie pokrywają się z moimi.
BAD Lubię chwilę po obudzeniu podrzemać i posłuchać radia, przy tym się nie da dostajesz patelnią w łeb i lecisz wyłączyć to paskudztwo.
BAD Zrezygnowałam z wizyt w klinice Polis i szukając pomocy udałam się do Lublina, gdzie zdiagnozowano u mnie wulwodynię.
BAD Na razie w sumie konkretnych badań nie robiłam bo jeszcze nie staram się o dziecko, ale jak już się zdecyduje to muszę chyba zmienić lekarza.
BAD Dowiedziałem się, że jeśli szczelina będzie mi przeszkadzać mam przyjść ponownie.
BAD Zamek w drzwiach taki ' niepewny ', futryna po przejściach, jakby ktoś kiedyś ' wjechał z buta ' do pokoju ( przez to nie czułam się tam w pełni bezpiecznie ).
BAD Pokoje sprzątane co 2 - 3 dni!
BAD Szczerze nie polecam!!!
BAD Myślę jednak że w tym roku byłem 1 raz i ostatni.
BAD Na plaży notorycznie wałęsały się psy i człowiek musiał uważać czy jakiś go nie ugryzie.
BAD Na miejscu doznalismy pierwszego szoku by dokonać opłaty za parking w kwocie 5 euro za dobę... gdzie w tui ( tam wykupowalismy hotel ) informowano nas, że parking jest bezpłatny.
BAD Szczerze odradzam, wlascicielka ma sie za wielka dame z milionamii a prawda jest taka ze hotel jest zbudowany z dotacji unijnych.
BAD Sauna sucha - tak designerska, ze nie ma gdzie usiąść.
BAD I do tego zmiana cennika w trakcie trwanie umowy!!!!
BAD cała obsługa " skacze " wokół gości którzy nie mają wykupionej opcji all a zdecydują sie zjeść w hotelowej restauracji ; podobnie było w barze gdzie goscie płacący gotówką dostawali co innego, inaczej podane i w pierwszej kolejności niż " reszta " ; żadnych animacji, niedobór leżaków przy basenach, itp...
BAD Hotel nie powinien mieć ich 5.
BAD Okolica bardzo brudna i zasmiecona, pełna bezdomnych psów.
BAD Przepisanie dawki leku i koniec wizyty.
BAD a tym czasem auto można zaparkować przy ulicy!!!
BAD Niedopracowane same rózgi do misy ( chociaż wyglądają bardzo profesjonalnie ) pozostawiają na dnie nierozmieszane ciasto lub krem ( zaznaczę ze posiadam inne miksery roboty które wypadają o niebo lepiej ) Najbardziej irytujące są jednak mieszadła silikonowe które powinny zbierać wszystko ze ścianek oraz spodu misy co bardzo przeszkadza a właściwie uniemożliwia podbijanie mas na * * * * * z jednoczesnym podgrzewaniem ( w tym celu to zakupiłem ) chyba że chcemy jajecznicę to wtedy są wręcz idealne.
BAD Fatalne łazienki z kamieniem, grzybem, smrodem.
BAD Dwa razy odwołana wizyta, a jak w końcu udało mi się z córką dostać to okazało się ze przyjmuje mnie asystentka, która nie potrafi odnieść się do wcześniejszych zaleceń pani doktor a mimo to kasuje zwyczajnie za wizytę.
BAD Absurdalne było też to, że nie można było często usiąść do konkretnego stolika poniewaz obsluga sobie przygotowała ten stolik na kolejny posiłek, na śniadaniu już sobie przygotowywali na lunch, na lunchu na kolację itd.
BAD Jestem tu drugi raz, i ostatni raz na dworze w nocy ok 4 stopni Celsjusza i nie ma ogrzewania, mimo prośby o jakiś grzejnik elektryczny, to niestety otrzymaliśmy odpowiedź od Pani że mamy koce i taki luksus w tym hotelu za 180 zł
BAD Później było już tylko gorzej …
BAD Nie rozpoznał u mnie astmy.
BAD Kobiety niech wam nawet do głowy nie przyjdzie iść na wizytę to tego człowieka!
BAD Słuchajcie, sprawa wygląda tak : Raczej odradzam zakupu 1 million woda po goleniu, bo zapach bardzo nikły... mimo, iż bardzo bardzo lubię ten zapach, bo mam 1 million wodę perfumowaną ( EDP ), kupioną przez internet ( sklep tylko z oryginalnymi produktami czyli te na ceneo.pl - najlepiej z wysoką ilością osób które kupują produkty ) - odradzam allegro.pl ( w perfumeriach takich sephora lub duglas nie sprzedają wody perfumowanej - dużo lepszej niż woda toaletowa ( EDT ), bo EDP ma dużo trwalszy zapach niż EDT ( czyli woda toaletowa ) - które to EDT sprzedają w tych perfumeriach.
BAD Nie polecam i proponuję trzymać się z dala od tej osoby, jeśli chcecie mieć lekarza zaufanego, który potrafi pomóc w różnych sytuacjach.
BAD Utrudnieniem może być bardzo słaba sień internetowa bezprzewodowa.
BAD Nie zmienia to jednak faktu, że mnie zbył niczym.
BAD W toalecie niedziałająca spłuczka z lejącą się całą noc wodą.
BAD Mam problemy z ładowaniem.
BAD Najgorsze w tym hotelu było że w szczytowym okresie ferii mazowieckim są robione remonty było głośno i pełno robotników poruszających po obiekcie.
BAD POZDRAWIAM PANI DOKTOR ; - /
BAD Tak jak w opinii powyżej zgodzę się, że wykonanie pozostawia wiele do jakości.
BAD Pełno stromych schodów i brak wind, jedzenie niesmaczne, niskiej jakości.
BAD Sorry ale jak na hotel 4 * to slabo.
BAD Wydałam ok 1300zł i na szczęście znalazłam innego lekarza stosujące Dry Needing igłami akupunkturowymi, gdzie po pierwszej wizycie i to tańszej jeszcze w tym samym dniu poszłam na zajęcia sportowe, a następnego dnia na długie zakupy po centrach handlowych.
BAD TEż z ARO.
BAD Poprawcie to.
BAD po godzinie czytania miałam większa wiedzę na temat małwodzia niż dr Łaganowski.
BAD Pani doktor na moją informację o tym, że bezskutecznie staramy się z mężem już rok o dziecko odpowiedziała, że to nie jest poradnia leczenia niepłodności.
BAD Jedzenie okropne, monotonne, typowo przygotowywane pod Brytyjczyków : wszystko smażone, na śniadanie fasola, na obiad i kolację frytki.
BAD Omijajcie szerokim łukiem to dziwne miejsce.
BAD W łazience pojemniki na mydło z płynem na samym dnie, starczyło na jeden prysznic.
BAD Osobiście nie skorzystamy już z oferty tego hotelu.
BAD W trakcie wizyty dowiedziałem się iż ząb jest pęknięty pionowo i nie nadaje się do leczenia oraz iż ząb należy usunąć.
BAD Nie ma możliwości wyłączenia głośnika.
BAD Koty na stołówce i w restauracji!
BAD Doktora Łazęckiego nie polecam.
BAD Przykładowe śniadanie to 2 plasterki sera, 2 plasterki wędliny i 3 plasterki pomidora.
BAD Bede musiał skorzystać z porady innego lekarza, ta wizyta nic nie wniosła, zwraca czasu i pieniędzy.
BAD Inna osoba pisała tu już, że wizyta trwala 3 minuty, a Pan Doktor uznał, że to nieprawda.
BAD Kolejny to problem z talerzami i sztućcami i szklankami na jadalni, jak przyszlo więcej ludzi to stalo się w kolejce czekając na nakrycie.
BAD Impreza u dr Nogala kosztowała mnie 340 zł i wolała bym przeznaczyć te pieniądze na inny cel.
BAD Takie podejście do zdezorientowanego pacjenta uważam za karygodne, bowiem po zleceniu badań od internisty okazało się, że mam ogromny niedobór wit.
BAD Nie polecam innych pokoi - widoki niekoniecznie ucieszą...
BAD Pokoje nie są sprzątane.
BAD Nie polecił bym tego radia, oczywiście wybiera każdy według własnego przekonania.
BAD Już o 22. 50 nic nie można zamówić z baru ( chyba, że za opłatą ).
BAD wizyta trwała 5min, miałam trochę większe oczekiwania wobec Pani Ani, szybkie rozpoznanie ' choroby ' recepta i do domu to za mało. .
BAD Kolejnego dnia chciał bym się wybrać na siłownie lecz niestety był remont i nikt nie raczył poinformować gości.
BAD Pan doktor u mnie zbagatelizował niepokojące objawy jakie wystąpiły po operacji i bez badań dodatkowych - wypisał do domu.
BAD Telewizja trochę szwankuje ( jest jakaś internetowa i czasem się przycina ) Niestety od przyszłego roku podobno ma być drożej ze względu na remont.
BAD Nasz pokój znajdował sie w rogu budynku, mimo to słychać było każde kroki i rozmowy z korytarza i innych pokoi.
BAD Wydałem 150zl plus dojazd.
BAD Stara suszarka wisząca na haczyku przez zawiniety kabel, tez nie napawa optymizmem, zwłaszcza ze po włączeniu w powietrzu unosił się zapach spalenizny.
BAD NIE PENSJONAT, WIĘC GDZIE JEST SALA ŚNIADANIOWA JAK W MARIOCIE W WARSZAWIE ?!
BAD Wykonałam na polecenie p. doktora niezliczone ilości kosztownych badań z których nic nie wynikało, zażywałam leki hormonalne, które powinnam brać pod okiem specjalisty w szpitalu na domiar złego każda kolejna wizyta u pana doktora trwałą 10 minut - bez wykonanego badania lekarskiego.
BAD Brak lobby i miejsc do wypoczynku.
BAD Generalnie odradzam bo jeśli lekarz czegoś nie jest pewien powinien się umieć przyznać do tego a nie udawać, że wie to karygodne tak niestety było w moim przypadku.
BAD Cale szczescie ze to byla tylko jedna noc bo nie mam zamiaru powtarzac przygody z tym hostelem.
BAD Byliśmy szczesciarzami, widziałem ludzi którzy oczekiwali po 8 godzin snując sie w upale po hotelu.
BAD Także zanim kupicie to polskie badziewie to zastanówcie się kilka razy - 3 letnia gwarancja na ten aku to śmiech na sali jak również serwis gwarancyjny.
BAD Na moja uwagę, iż katar ostatnio trwał długo i przerodzil się w wysiekowe zapalenie ucha i prośbę by poleciła coś na przyspieszenie jego ustapieniu, usłyszałam że nie ma potrzeby na tym etapie i oczywiście zawsze może tak się zdarzyć że katar spowoduje komplikacje.
BAD Jedzenia bardzo mało i niesmaczne A między obiadem A kolacja nie ma co jeść.
BAD We wszystkich informatorach widnieje godzina 22. 00.
BAD Wejście na teren kempingu nie był w żaden sposób monitorowany, nie czułam się więc bezpiecznie zostawiając tam rowery i namiot.
BAD Moim zdaniem jak jechac na wczasy to lepiej cos dorzucic i miec swiety spokoj i good time a nie ogladac to kiche co tam sie odprawia.
BAD Właściciel nie zaproponował jakiegokolwiek alternatywnego rozwiązania i rekompensaty, nie otrzymaliśmy nawet rachunku.
BAD Z wad jeszcze wymienił bym zacinanie się Youtuba.
BAD U mnie zaczęło się od tego że przesyłka szła do serwisu 13 dni bo takiego sobie przewoźnika wybrali... ale to był dopiero początek!
BAD Mało cierpliwy i nie ma moim zdaniem dobrego podejścia do pacjentek.
BAD Niestety, nie wyleczył mnie do końca - brakło mu pomysłów.
BAD Gburowaty, bardzo obraża ludzi, zupełnie niepotrzebne i przykre słowa w stosunku do drugiego człowieka, który przychodzi po poradę.
BAD Ocena " Słaby " za to, że powiedziała że prostatitis jest nieuleczalne co jest nieprawdą.
BAD Dodatkowo tuż po naszym przybyciu do pokoju była zupełnie pusta i nie mogliśmy napić się nawet wody po długiej podróży.
BAD Prócz skibusów brak komunikacji.
BAD Wino rozcieczone woda Jedzenie monotonne.
BAD Personel zupełnie niekompetentny!
BAD Obiadokolacji nie polecam.
BAD Pokoje niezbyt ładne.
BAD O wszystko trzeba się samemu pytać.
BAD Wywiad zajął Jej dłużej niż sama rozmowa o problemie i jeszcze usłyszał em, ze byłem za długo.
BAD Hotel nie ma parkingu i autokar musiał stać bardzo daleko.
BAD Winien jest profil miski.
BAD Z niekompletnym zrostem lekarka stwierdziła iż jestem zdrowy i nie widzi przeciwwskazań do wykonywania nawet ciężkich prac fizycznych.
BAD Pani doktor niestety kompletnie nie potrafiła mnie znieczulić, przez co w sumie nietrudne leczenie przerodziło się w koszmar dla mnie i orkę na ugorze dla niej.
BAD Szelki to tylko same paski parciane z kawałkiem skaju na biodro.
BAD W basenie zimna woda, w jacuzzi też.
BAD Mi zupełnie nie przypadł do gustu.
BAD Szkoda jedynie że jest na ciepło, przez co nie działa po stuknięciu paznokciem, długopisem czy rysikiem.
BAD do hoteu prowadzą schody, a więc ciężkie walizki trzeba nosić samodzielnie - brak podjazdu albo windy.
BAD Obiadokolacja w cenie 50 zl od osoby straszna, dania podane zimne, ociekajace tluszczem.
BAD Otoczenie ośrodka smutne, niezadbane.
BAD Mam taki bidon i jestem rozczarowana.
BAD Np. Odmawia wykonania odpłatnie USG jajników, bo coś jej się ubzdurało.
BAD Cena nie adekwatna do komfortu jaki oferuje hotel.
BAD Niestety już pierwszego dnia w recepcji czekała na nas bardzo niemiła niespodzianka.
BAD Prolaktyna nadal była ponad normą, natomiast pani ordynator przy wypisie przepisała mi sterydy ( mówiła zeby nie czytać ulotki bo moge sie wystraszyc ), jakiś środek na łaknienie ( w związku z moją rzekomą anoreksją ) i branie w dalszym ciągu norprolaku ( jak się później okazało nie powinno się go brać dłużej niż pół roku ).
BAD Sale prelekcyjne porażka, były prowadzone równocześnie dwa szkolenia poproszono o wyłączenie mikrofonów bo wzajemnie się zagłuszaliśmy.
BAD Nie odbiera telefonów, mimo, że wcześniej każe siebie informować o stanie zdrowia dziecka.
BAD Po otwarciu drzwi pokoju uderzał nieprzyjemny zapach pleśni i wilgoci.
BAD Nie było by tyle nie przespanych nocy, wątpliwości i mniej stresu.
BAD Montaz tej kamerki na monitorze graniczy z cudem bo uchwyt który posiada jest tak zaprojektowany że nie nadaje sie do niczego, Aby ją stabilnie zamontowac zamówiłem rzepy do podklejenia bo inaczej jest nie stabilna i co chwila trzeba ja poprawiac bo się sama obraca razem z mocowaniem.
BAD Dodam, że przed wizytą zrobiłam już część badań, z których dobry, doświadczony lekarz potrafił by wysunąć jakiekolwiek, chociaż drobne wnioski.
BAD Generalnie od 9. 00 do 22. 00 - 23. 00 ( wtedy kończyły się ostatnie imprezy ) cały czas coś gra...
BAD Stanowczo odradzam
BAD z wizyty u tego lekarza wyszłam załamana.
BAD A wszystko to okraszone dość wysokimi wymaganiami sprzętowymi jeśli zamierzamy grać na pełnych detalach graficznych.
BAD Zdecydowanie jednak nie chciała bym znaleźć się na miejscu tych ludzi.
BAD Niestety widok był z widokiem na parking i na miejsce gdzie ludzie podczas imprezy wychodzą palić papierosy.
BAD Pytania podczas laborki potrafią byc ciężkie, można wylecieć na sam koniec laboratorium, ale z reguły 3, może 4 grupy bedą zapytane podczas całych zajęć.
BAD Szatnie na dole kolo basenu co może być kłopotliwe dla niektórych.
BAD Male pokoje w ktorych widac ząb czasu.
BAD Nakłada i rozprowadza się idealnie ale najmniejszy zaciek kończy się ciemną plamą która przy próbie zamalowania robi się czarna i niemożliwa do usunięcia, gdyż w pomalowanym miejscu zarówno po wyschnięciu jak i przed wciągana jest w drewno kolejna warstwa impregnatu co doprowadza do sytuacji w której miejsce takie z kolejną warstwą robi się coraz ciemniejsze aż do całkowicie czarnego.
BAD Pani doktor w trakcie usg stwierdziła że faktycznie jestem w ciąży a gdy obydwoje z mężem popłakaliśmy się ze szczęscia zapytała z oburzeniem " czego pani ryczy ? ".
BAD Sprzætanie bardzo kiepsko.
BAD Ocena ogólna : UNIKAC : x : x
BAD Nie polecam!!!!!!!!!!!!!!!!!!!
BAD Jest to jawna niesprawiedliwość.
BAD Znajomi zaproponowali, żebym zostawił bagaże i przeszedł sie do hotelu obok na co Jakiś buc ( domniemam że właściciel ) siedzący w holu powiedział, że w żadnym wypadku nie mogę zostawić bagaży - mam je zabierać i wyjść... co z największą przyjemnością uczyniliśmy.
BAD Łazienka ze zniszczonym brodzikiem, zatkany brodzik, brudne fugi na płytkach, kontakty nieprzykręcone do ściany po malowaniu, na korytarzu czuć palonymi papierosami w recepcji, W opisie hotelu jest opcja śniadań lecz na miejscu okazuje się, że ich nie ma.
BAD z dnia na dzien bez słowa wyjasnien odwołali moją rezerwacje a gdy dzwoniłam zeby wyjaśnić dlaczego taka sytuacja zaistniała włascicielka była oprysklliwa i niemiła mówiąc " ze ją to nie obchodzi gdzie teraz znajde nocleg to mój kłopot itp " Już z wczesniejszych rozmów telefonicznych dało sie odczuć że to niezbyt przyjazni ludzie... odniosłam wrażenie że tak naprawde to łaskę mi robią że dają mi nocleg.
BAD Przyciski panelu sterowania głośno pikają po naciśnięciu.
BAD W pokoju przy łóżku nie ma żadnego gniazdka, żeby móc przez noc podładować telefon.
BAD Ten zestaw jest kompletnie do niczego.
BAD Po wejściu do gabinetu Pani Janiszewska nie pozwolił zamknąć drzwi wejściowych z tego powodu cała rozmowa była słyszana przez pacjentów na korytarzu.
BAD to sa te trafne zaściankowe diagnozy.
BAD Moje ostatnie dwie wizyty odbyły się niezgodnie z podaną wcześniej informacją i ostatecznie wyniosły mnie o 150 zł więcej niż byłam informowana wcześniej, a specjalnie pytałam kilka razy o koszt. .
BAD Zarówno poziomem wiedzy medycznej, jak i kultury osobistej.
BAD Pani dr nie stwierdziła nic niepokojącego a tym bardziej już podejrzenia tarczycy, bo to przecież ( mało prawdopodobne jej zdaniem ).
BAD Ale do dźwięku bym się przyczepił.
BAD Pająki w pokoju i szerszenie.
BAD A to w nieprzyjemny sposób " przesłuchiwał " na porannej wizycie śmiertelnie zmęczoną lekarkę przekazującą dyżur.
BAD Przypuszczam, że wszystkie jego pacjentki, które urodziły dzieci w trakcie leczenia u niego po prostu miały szczęście.
BAD Co jest najdziwniejsze ?
BAD Rozczarowała mnie również okolica.
BAD Nieprofesjonalna obsługa w barze ( pozdrowienia dla Dawida ), słaby DJ ( puszcza tytuły, a nie rytmy do tańczenia, nie rozumie co się do niego mówi ) nic dziwnego, że klub świeci pustkami, a jedyną sensowną atrakcją jest bowling w przesadnej cenie 80 zł / h ( jak rozumiem za dużo chętnych z powodów wymienionych wcześniej ) Kolejną sprawą jest sposób płatności na pokój, żeby nie nosić karty kredytowej lub gotówki np do basenu.
BAD Niestety miałam nieprzyjemność zetknąć się z nim znów w tej przychodni, gdy zgłosiłam się w trybie pilnym, z powodu zapalenia twardówki i błony naczyniowej oka.
BAD Zero w nich energii.
BAD Zimna woda ( jak sie odkreci prysznic na pol godziny, to leci ciepla ), lazienka na 2 pokoje ( w cenniku nazywaja to " studio " ), w pokoju zimno ( dostalem farelke, zeby sie dogrzac ), sniadanie kiepskie ( a calosc dopelnia kawa rozpuszczalna z automatu jak na dworcu ), Internet woooolny ( o ile akurat jest, bo rwie sie co chwile ).
BAD Jedyne, co mnie zaskoczyło, to pewne nieprzemyślane elementy budowy.
BAD Pokój z widokiem na północ 0 gór.
BAD Nie polecam korzystania z opcji obiadokolacji, zaserwowano zupę pieczarkową a na danie główne gołąbek z ziemniakami.
BAD Szkoda czasu i nerwów to rada dla wybierających się do profesora.
BAD Ogolnie butelki nie zostaly wysterelizowane, w toalecie w efekcie czego dziecko dostalo plesniawek.
BAD Niestety dzisiaj przyjechałam z mężem na wyczekiwanyw weekendw hotelu Diament i niestety już na samym początku okropne rozczarowanie - otrzymaliśmy pokój z widokiem na betonowy mur...
BAD Jednak pokój pozostawial wiele do życzenia.
BAD Również zmieszane zapachy wielu potraw nie zachęcały do dłuższego pobytu.
BAD Gondola i spacerówka to jest po prostu koszmar.
BAD Jak tylko dostaje zasilanie wpada w taki rezonans, że zagłusza dwa inne dyski + 2 wentylatory w obudowie, wentylator i buczenie zasilacza, chłodzenie procesora oraz dwa wentylatory grafiki razem wzięte.
BAD Jeśli ktoś liczy na basen do pływania z małymi dziećmi ( poniżej 4 lat ), musi wiedzieć że w hotelowym basenie nie mogą się kąpać dzieci.
BAD Ale jak juz sa komplikacje to chyba nie jest najlepszy.
BAD Oszczędzają ( a raczej dziadują ) na wszystkim, co się da.
BAD Po drugie : jedzenie było PASKUDNE!
BAD Żeby móc korzystać z pokoju musieliśmy go najpierw porządnie posprzątać.
BAD Kolejny raz i kolejne WIELKIE rozczarowanie, jeśli chodzi zarówno o czystość, obsługę, jedzenie i ogólne wrażenie.
BAD Jak na specjalistę w dziedzinie medycyny estetycznej, trochę slabe podejście do problemu.
BAD Badanie trwało dosłownie moment - dr nie zlecił natychmiastowego badania moczu, czego wymagał mój stan ( ból pochwy i pęcherza ).
BAD Jeden raz był makaron ( przypalony, z papryką chilli ).
BAD Na pewno nikomu nie polecam tego hotelu!
BAD Szklanki brudne stały nie zmieniane od niedzieli, sama je wynosilam i prosiłam o czyste, kabina prysznicowa nie myta.
BAD ZaleceniaKupując wczasy z biura Rainbow dobrze się zastanów czy umowa gwarantuje komfort jakiego oczekujesz sugerując się standardem pokoju w hotelu.
BAD nie warto wydawać pieniądze na niepotrzebne leki i tracic czas.
BAD Udałem sie do toalety gdzie zauważyłem ze zabieg spowodował krwawienie.
BAD Dodatkowo był niepomocny, niezainteresowany wyjaśnieniem problemu.
BAD Otrzymałem stanowczą odpowiedź, że mogę tylko otrzymać badania zgodne z dokumentacją pacjenta.
BAD Mała po tym dostała jeszcze większych kolek.
BAD Jedyny minus to bardzo wysokie ceny jak na Polskę 700 plan za nocleg, 1700 pln za 4 - osobowa salkę, 180 pln za steka z antrykotu z kieliszkiem wina, czy piwo za 20pln.
BAD Oczywiście nie obwiniam Doktora Hodery o to co się stało, bo to było " dziełem " natury, ale bardzo nie spodobało mi się Jego podejście do utracenia ciąży.
BAD Mało przyjemne.
BAD Może wystrój nie do końca w moim guście.
"""

In [ ]:
model_name = "allegro/herbert-base-cased"
device = 'cuda'

tokenizer_herbert = AutoTokenizer.from_pretrained(model_name)
model_herbert = AutoModel.from_pretrained(model_name).to(device)

text = 'Bardzo lubię lody malinowe z bitą śmietaną.'

token_ids = tokenizer_herbert(text, return_tensors='pt')['input_ids'][0]

print ([tokenizer_herbert.decode(idx) for idx in token_ids])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/907k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/556k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.sso.sso_relationship.bias', 'cls.sso.sso_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['<s>', 'Bardzo', 'lubię', 'lody', 'mali', 'nowe', 'z', 'bi', 'tą', 'śmietan', 'ą', '.', '</s>']


In [ ]:
model_name_papuga = 'flax-community/papuGaPT2'

tokenizer_papuga = AutoTokenizer.from_pretrained(model_name_papuga)
model_papuga = AutoModelForCausalLM.from_pretrained(model_name_papuga).to(device)

def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label

def sentence_prob(sentence_txt):
    input_ids = tokenizer(sentence_txt, return_tensors='pt')['input_ids'].to(device)
    with torch.no_grad():
        output = model(input_ids=input_ids)
        log_probs = log_probs_from_logits(output.logits[:, :-1, :], input_ids[:, 1:])
        seq_log_probs = torch.sum(log_probs)
    return seq_log_probs.cpu().numpy()

tokenizer_config.json:   0%|          | 0.00/208 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/888k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/547k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.54M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/864 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [ ]:
def get_probs(prompt, ending_positive, ending_negative):
    prompt_positive = prompt + ending_positive
    prompt_negative = prompt + ending_negative
    prob_positive = sentence_prob(prompt_positive).item()
    prob_negative = sentence_prob(prompt_negative).item()
    return prob_positive, prob_negative

In [ ]:
get_probs("Wacław łowi ryby", ". Polecam.", ". Nie polecam")

(-47.64659118652344, -48.674015045166016)

In [ ]:
def representation(L):
    txt = ' '.join(L)
    input_ids = tokenizer_herbert(txt, return_tensors='pt')['input_ids'].to(device)
    embeddings = model_herbert.embeddings.word_embeddings(input_ids)[0, :, :].mean(dim=0).detach()
    # prompt_positive_1 = txt + ". Polecam."
    # prompt_negative_1 = txt + ". Nie polecam"
    # prob_positive_1 = sentence_prob(prompt_positive).item()
    # prob_negative_1 = sentence_prob(prompt_negative).item()
    probs = torch.tensor([*get_probs(txt, ". Polecam.", ". Nie polecam."), *get_probs(txt, ". Pozytywnie oceniam.", ". Negatywnie oceniam."), *get_probs(txt, ". Super.", ". Złe.")]).to(device)
    return torch.cat((embeddings, probs))

In [ ]:
representation("Wacław łowi ryby".split(" ")).shape

torch.Size([774])

In [ ]:
def label_data():
  X, Y = [], []
  lines = s.splitlines()
  random.shuffle(lines)
  for line in lines:
      if not line:
        continue
      L = line.split(" ")
      # print(L[0], L[0] == 'BAD')
      y = 0 if L[0] == 'BAD' else 1
      y = torch.tensor([y], dtype=torch.float32)
      x = representation(L[1:])
      Y.append(y)
      X.append(x)
  return torch.stack(X), torch.stack(Y)

In [ ]:
X, Y = label_data()

In [ ]:
indices = torch.randperm(X.size(0))
X = X[indices]
Y = Y[indices]
N = len(X)
test_size = N // 4
train_size = N - test_size

X_train = X[:train_size].cpu()
Y_train = Y[:train_size].cpu()
X_test = X[train_size:].cpu()
Y_test = Y[train_size:].cpu()

In [ ]:
X_train.device, Y_train.device, X_test.device, Y_test.device

(device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'))

In [ ]:
clf = LogisticRegression(random_state=0).fit(X_train, Y_train)

print ('Train accuracy:', clf.score(X_train, Y_train))
print ('Test accuracy:', clf.score(X_test, Y_test))

Train accuracy: 0.9233333333333333
Test accuracy: 0.94


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
clf = LogisticRegression(random_state=0).fit(X_train, Y_train)

print ('Train accuracy:', clf.score(X_train, Y_train))
print ('Test accuracy:', clf.score(X_test, Y_test))

Train accuracy: 0.9333333333333333
Test accuracy: 0.93


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
